# Lab 1: Regular Expressions and Text Normalisation

### Aims

- Install the required libraries and refamiliarise yourself with Python and Jupyter notebooks
- Get familiar with some basic NLTK tools for preprocessing text
- Understand regular expressions
- Carry out text normalisation steps

### Outline

- Getting started: how to set up your environment, Jupyter notebooks introduction
- Acquiring raw text data
- Regular expressions
- Text normalisation

### How To Complete This Lab

Read the text and the code then look for 'TODOs' that instruct you to complete some missing code. You don't have to stick rigidly to the lab -- feel free to explore other methods and data to help you understand what's going on or to try out new methods that go beyond this lab.

Aim to work through the lab during the scheduled lab hours. You can also post your questions to our Team's QA channel throughout the week.

The labs _will not be marked_. However, they will prepare you for the coursework, so try to keep up with the weekly labs and have fun with the exercises!

### Additional Exercises

If you would like to do more lab exercises or would like an alternative explanation, please see Chapters 1-3 in the NLTK book, which goes into more detail than we do here. https://www.nltk.org/book/


## Getting Started

### Setting up your environment

We recommend using `conda` to create an environment with the correct versions of all the packages you need for these labs. You can install either Anaconda or Miniconda, which will include the `conda` program.

We provide a .yml file alongside this notebook that lists all the packages you will need, and the versions that we have tested the labs with. You can use this file to create your environment as follows.

1. Open a terminal. Use the command line to navigate to the directory containing this notebook and the file `dialogue_and_narrative.yml`. You can use the command `cd` to change directory on the command line.

1. Run the conda program by typing `conda env create -f dialogue_and_narrative.yml`, then answer any questions that appear on the command line.

1. Activate the environment by running the command `conda activate dialogue_and_narrative`.

1. Make kernel available in Jupyter: `python -m ipykernel install --user --name=dialogue_and_narrative`.

1. Relaunch Jupyter: shutdown any running instances, and then type `jupyter lab` or `jupyter notebook` into your command line, depending on whether you prefer the full Jupyter lab development environment, or the simpler Jupyter notebook.

1. Find this notebook and open it up again.

1. Go to the top menu and change the kernel: click on 'Kernel'--> 'Change kernel' --> data_analytics.

You should now be ready to go!

The core libraries we will be using in this unit are:

- [Datasets](https://huggingface.co/docs/datasets/), produced by HuggingFace, is a hub for lots of interesting text datasets.
- [NLTK](https://www.nltk.org), a comprehensive NLP library.
- [Scikit-learn](https://scikit-learn.org/stable/user_guide.html), for machine learning and classifier evaluation.
- [Transformers](https://huggingface.co/docs/transformers/), also produced by HuggingFace, this library wraps up many pretrained deep neural networks, which we will look at later in the unit.

The libraries above have good documentation, which is available either online (links above) or via Python itself, e.g. `help(numpy.array)` in the Python interpreter.

### Refreshers for Python and Jupyter

**Skip this part if you're already familiar with Python and Jupyter notebooks.**

This lab assumes you have used Python and Jupyter Notebooks before.

For an introduction or refresher on Python, see the [Introduction to Python lab](https://github.com/UoB-COMS21202/lab_sheets_public/tree/master/lab_1) or the University of Bristol [Beginning Python](https://milliams.gitlab.io/beginning_python/) course. If you are a beginner with Python, you might also like to look at Chapter 1 in the NLTK book, which also provides a guide for "getting started with Python": https://www.nltk.org/book/

You will need to use Python 3, not Python 2. Specifically Python 3.6 or newer is recommended.

The labs will be run on [Jupyter Notebook](http://jupyter.org/), an interactive coding environment embedded in a webpage supporting various programing languages (Python, R, Lua, etc.) through the concept of kernels.

It allows you to enrich your code with complex comments formatted in Markdown and $\LaTeX$, as well as to place the results of your computation right below your code.

Notebooks are organised in cells which can contain either code (in our case, this will be Python code) or text, which can be easily and nicely formatted using the Markdown notation.

To edit an already existing cell simply double-click on it. You can use the toolbar to insert new cells, edit and delete them (or use keyboard shortcuts which are very handy to speed up coding).

Cells can be run, by hitting `shift+enter` when editing a cell or by clicking on the `Run` button at the top. Running a Markdown cell will simply display the formatted text, while running a code cell will execute the commands executed in it.

**Note**: when you run a code cell, all the created variables, implemented functions and imported libraries will be then available to every other code cell. However, it is commonly assumed that cells will be run sequentially in terms of prerequisites. To reset all variables and functions (for debugging) simply click `Kernel > Restart` from the Jupyter menu.

#### Markdown

Markdown cells allow you to write fancy and simple comments: all of this is written in Markdown - double click on this cell to see the source. Introduction to Markdown syntax can be found [here](https://daringfireball.net/projects/markdown/syntax).

As Markdown is translated to HTML upon displaying it also allows you to use pure HTML: more details are available [here](https://daringfireball.net/projects/markdown/syntax#html).

Finally, you can also display simple $\LaTeX$ equations in Markdown thanks to `MathJax` support. For inline equations wrap your equation between `$` symbols; for display mode equations use `$$`.


# 2. Doc2Dial Dataset

The Doc2dial dataset was introduced for a 'shared task', where teams from different institutions compete in building a dialogue system. The dataset contains dialogues between a user and a customer service agent working for the US department of motor vehicles. The task is broken into two steps: first, retrieve relevant information from a document to answer a user's query; then, use this information to formulate a response as a line of dialogue. More on the task here:
https://doc2dial.github.io/workshop2021/shared.html

The raw data is available [here](https://doc2dial.github.io/file/doc2dial_v1.0.1.zip) but we will use a data loader class from the [HuggingFace datasets library](https://huggingface.co/docs/datasets/loading_datasets.html) to load it.


In [1]:
import json


def generate_examples(filepath):
    """
    Adapted from https://huggingface.co/datasets/doc2dial/blob/main/doc2dial.py#L226
    """
    with open(filepath, encoding="utf-8") as f:
        data = json.load(f)
        for domain in data["dial_data"]:
            for doc_id in data["dial_data"][domain]:
                for dialogue in data["dial_data"][domain][doc_id]:
                    x = {
                        "dial_id": dialogue["dial_id"],
                        "domain": domain,
                        "doc_id": doc_id,
                        "turns": dialogue["turns"],
                    }

                    yield dialogue["dial_id"], x


dataset = [
    *generate_examples(
        # The filepath of the training data from the repository:
        # https://github.com/doc2dial/sharedtask-dialdoc2021
        "../../sharedtask-dialdoc2021/data/doc2dial/v1.0.1/doc2dial_dial_train.json",
    )
]

In [2]:
from pprint import pprint

pprint(f"The dataset has {len(dataset)} instances")

pprint("An example instance: ")
pprint(dataset[2342])

'The dataset has 3474 instances'
'An example instance: '
('92db4f3c68ab3fb2851f4a559d9c2d1e',
 {'dial_id': '92db4f3c68ab3fb2851f4a559d9c2d1e',
  'doc_id': 'Temporary Disability Rating After Surgery Or Cast | Veterans '
            'Affairs#1_0',
  'domain': 'va',
  'turns': [{'da': 'query_condition',
             'references': [{'label': 'precondition', 'sp_id': '28'}],
             'role': 'user',
             'turn_id': 1,
             'utterance': 'Hello I need information on How do I get these '
                          'disability rating benefits?'},
            {'da': 'respond_solution',
             'references': [{'label': 'solution', 'sp_id': '29'},
                            {'label': 'solution', 'sp_id': '30'}],
             'role': 'agent',
             'turn_id': 2,
             'utterance': 'To get these benefits, you must file a disability '
                          'compensation claim. Find out how to file a '
                          'disability compensation claim'

Look at the formatting of the output -- it looks like a nested set of Python dictionaries and lists. We can access the different fields in a single data sample as if reading from a Python dictionary.

For our lab this week, we will need some examples of dialogue written by a user. Let's get some from the training set of Doc2Dial.

**_TODO 1.1:_** get a list containing all the user utterances from 100 different conversations. Name the list 'utterances'. Hint: look at the output from the previous cell to see where the utterances are stored.


In [3]:
utterances = [
    turn["utterance"] for item in dataset[:100] for turn in item[1]["turns"]
]

pprint(len(utterances))
pprint(utterances[:10])

1195
['Hello, I forgot o update my address, can you help me with that?',
 'hi, you have to report any change of address to DMV within 10 days after '
 'moving. You should do this both for the address associated with your license '
 'and all the addresses associated with all your vehicles.',
 'Can I do my DMV transactions online?',
 'Yes, you can sign up for MyDMV for all the online transactions needed.',
 'Thanks, and in case I forget to bring all of the documentation needed to the '
 'DMV office, what can I do?',
 "This happens often with our customers so that's why our website and MyDMV "
 'are so useful for our customers. Just check if you can make your transaction '
 "online so you don't have to go to the DMV Office.",
 'Ok, and can you tell me again where should I report my new address?',
 "Sure. Any change of address must be reported to the DMV, that's for the "
 'address associated with your license and any of your vehicles.',
 'Can you tell me more about Traffic points and thei

# 2. Regular Expressions

Next, we are going to experiment with building a simple chatbot using regular expressions, which are an important NLP tool -- not everything needs machine learning! The aims are to get familiar with regular expressions and to see some limitations of rule-based approaches.

Many text processing systems make use of regular expressions, which are a language for specifying sets of strings. We can use regular expressions to define a pattern to search for in a corpus of text and retrieve all the occurrences of that pattern. We can also use regular expressions to replace one text pattern with another. Regular expressions are therefore used in various NLP systems, e.g., to implement tokenisation or extract features for a classifier by looking for specific patterns. They can often be used to build a simple baseline for tasks like text classification before developing a machine learning solution.

## 2.1 Search

Suppose we want to identify when the user asks the agent whether they can do something. We can start by finding occurrences of the word "can":


In [4]:
from pprint import pprint
import re

matches = [
    match for utterance in utterances for match in re.findall(r"can", utterance)
]

# Print the unique items of the list.
pprint(set(matches))

# Print the length of the list.
pprint(len(matches))

{'can'}
289


The regular expression above searches for an exact match, which means we will miss the cases where 'can' is capitalised. Fortunately, regular expressions define a set of strings by specifying a pattern, allowing us to generalise our search and retrieve a set of many different strings that match the pattern. Let's improve our search step by step to retrieve retrieve both capitalised and lower case occurrences of the word.

To do this, we will use the _disjunction_ capability of REs. The disjunction of two or more characters is written using square brackets. The regular expression now defines a set of strings, including this disjunction.


In [5]:
from pprint import pprint
import re

matches = [
    match
    for utterance in utterances
    for match in re.findall(r"[Cc]an", utterance)
]

# Print the unique items of the list.
pprint(set(matches))

# Print the length of the list.
pprint(len(matches))

{'can', 'Can'}
345


This has given us a list of matches in the variable `all_matches`, which all contain the string 'can', but not the complete phrases containing this word, which isn't very useful!

In the next cells, we will expand our pattern to match the word following 'can' as well. To do this, we need to broaden the disjunction by using some special characters in the RE string:

- Match any lower case letter: 'a-z'
- Repetition: Match zero or more repetitions of the preceding RE: '\*'

There are lots of other special characters -- for a complete list of special characters, see https://docs.python.org/3/library/re.html#regular-expression-syntax.

The code below retrieves strings containing 'can' followed by the first letter of the next word.

**_TODO 2.1:_** Using the special characters mentioned above, modify the RE below to retrieve the complete following words.


In [6]:
from pprint import pprint
import re

matches = [
    match
    for utterance in utterances
    for match in re.findall(r"[Cc]an [a-z]*", utterance)
]

# Print the unique items of the list.
pprint(set(matches))

# Print the length of the list.
pprint(len(matches))

{'Can ',
 'Can a',
 'Can another',
 'Can i',
 'Can my',
 'Can there',
 'Can you',
 'can ',
 'can actually',
 'can also',
 'can always',
 'can appeal',
 'can apply',
 'can ask',
 'can assure',
 'can avoid',
 'can be',
 'can begin',
 'can but',
 'can check',
 'can clearly',
 'can come',
 'can confirm',
 'can contact',
 'can continue',
 'can do',
 'can find',
 'can focus',
 'can get',
 'can give',
 'can go',
 'can he',
 'can help',
 'can i',
 'can inform',
 'can issue',
 'can issues',
 'can it',
 'can know',
 'can mail',
 'can make',
 'can miss',
 'can my',
 'can not',
 'can only',
 'can opt',
 'can or',
 'can perfectly',
 'can prevent',
 'can proceed',
 'can read',
 'can register',
 'can renew',
 'can resolve',
 'can schedule',
 'can search',
 'can see',
 'can send',
 'can sign',
 'can solve',
 'can still',
 'can submit',
 'can supply',
 'can surely',
 'can take',
 'can tell',
 'can that',
 'can the',
 'can then',
 'can there',
 'can transfer',
 'can turn',
 'can use',
 'can verify',
 'c

Suppose we want to extract only the words that follow 'can' -- how can we separate the following word from 'can' itself?

Here, we can use _groups_, to group the characters to match the characters of 'can' into one group, and the characters that match the following word into another group. In RE syntax, parentheses '(...)' encapsulate _groups_. A group is a regular expression nested within a larger RE. Groups are especially useful because you can apply special characters such as \* to expressions inside a group.

The findall function returns the matches as tuples of length N, where N is the number of groups in the expression.

**_TODO 2.2:_** Modify the expression below to extract a list of words that follow 'can'. Hint: what happens if we remove one set of parentheses?


In [7]:
from pprint import pprint
import re

matches = [
    match.group("word")
    for utterance in utterances
    for match in re.finditer(r"(?P<can>[Cc]an) (?P<word>[a-z]*)", utterance)
]

# Print the unique items of the list.
pprint(set(matches))

# Print the length of the list.
pprint(len(matches))

{'',
 'a',
 'actually',
 'also',
 'always',
 'another',
 'appeal',
 'apply',
 'ask',
 'assure',
 'avoid',
 'be',
 'begin',
 'but',
 'check',
 'clearly',
 'come',
 'confirm',
 'contact',
 'continue',
 'do',
 'find',
 'focus',
 'get',
 'give',
 'go',
 'he',
 'help',
 'i',
 'inform',
 'issue',
 'issues',
 'it',
 'know',
 'mail',
 'make',
 'miss',
 'my',
 'not',
 'only',
 'opt',
 'or',
 'perfectly',
 'prevent',
 'proceed',
 'read',
 'register',
 'renew',
 'resolve',
 'schedule',
 'search',
 'see',
 'send',
 'sign',
 'solve',
 'still',
 'submit',
 'supply',
 'surely',
 'take',
 'tell',
 'that',
 'the',
 'then',
 'there',
 'transfer',
 'turn',
 'use',
 'verify',
 'view',
 'visit',
 'we',
 'you'}
299


This is starting to seem more useful -- we've retrieved a set words that are used alongside a term of interest. What we really want is to extract the whole context of these words, i.e., the sentences or phrases they are contained in. For this we may need a few more special characters:

- Complement, match any character except the specified ones: '[^A]'
- New line: '\\n'
- Escape: e.g., '\\?', '\\'. Using the backslash in front of special characters means that they are not interpreted as special chracters but are treated literally, in this case as a question mark or full stop.

The code below uses the complement special character to match the next character after 'can' as long as it is not a punctuation mark or newline.

**_TODO 2.3:_** Modify the expression below so that it retrieves the complete phrase including the word 'can', from the start of the utterance or the previous punctuation mark, until the next punctuation mark or newline.

**_TODO 2.4:_** In the cell below, in the line that computes 'all*matches = \[...', what do the square brackets '\[...\]' do? Hint: \_list comprehension*


In [42]:
from pprint import pprint
import re

matches = [
    (match.group("before") + match.group("can") + match.group("after")).strip()
    for utterance in utterances
    for match in re.finditer(
        r"(?P<before>[^\.\?!;:,\n]*)(?P<can>[Cc]an )(?P<after>[^\.\?!;:,\n]*)([\.\!\?\n;:,])",
        utterance,
    )
]

# Print the unique items of the list.
pprint(set(matches))

# Print the length of the list.
pprint(len(matches))

{'Before going to a DMV Office you should see if your transaction can be '
 'performed online',
 'But take into consideration that the card will be rejected if the DMV '
 'barcode reader can not scan the barcode',
 'But what can I do with my insurance from another state',
 'But you must know that you can only go for it after the DMV TVB Default UNit '
 'has denied your Application to Reopen a Default Conviction',
 'Can I avoid to pay the fine after filing an appeal',
 'Can I do it by mail',
 'Can I do my DMV transactions online',
 'Can I file an appeal by mail',
 'Can I fill in some of the paperwork online',
 'Can I get a refund for the first year of my registration after receiving a '
 'refund for the second year',
 'Can I give my plates to another person',
 'Can I just notify the United States Postal Service about a change of address '
 'or do I have to update this info with the DMV too',
 "Can I just write my new address in the back of my license if I haven't "
 'updated my address 

### 2.2 Substitution

We can also use regular expressions to replace one string with another, i.e., _substitution_. This is extremely useful for implementing text preprocessing steps, which clean up and format the text so that it can be processed by other methods such as text classifiers. We can even use it to implment simple chatbots!

In Python, we can use the re.sub() function to replace one regular expression with an other. re.sub() takes three arguments:

- The first argument specifies the search expression, which is the expression to match in the input text
- The second defines the replacement pattern we should replace the search expression with
- The third is the input text that we want to apply the subtitution to.

As before, the search expression can match _groups_ of characters. The replacement pattern can also include these groups of characters by referring to them using special variables. We use '\1' for the first group, '\2' for the second, etc.

The example below shows how a dialogue system can use substitution to personalise a greeting:


In [9]:
# An example of user input.
utterance = "Hello, my name is Ada Lovelace."
pprint("[user] " + utterance)

# A regular expression to find the user's name from the input and generate a response.
re_name = r".*[Mm]y name is (?P<name>[a-zA-Z ]*)([\.\!\?,])(.*)"

# Try to find the user's name.
match = re.match(re_name, utterance)

if match:
    # Print the user's name.
    pprint(match.group("name"))

    # Print the chatbot's response.
    pprint("[chatbot] " + re.sub(re_name, r"Hello \1!", utterance))
else:
    pprint("[chatbot] I do not understand.")

'[user] Hello, my name is Ada Lovelace.'
'Ada Lovelace'
'[chatbot] Hello Ada Lovelace!'


Note: Only the search expression match will be substituted by the replacement expression.

Let's use regular expression substitutions to create our first dialogue system! In 1966, a famous chatbot, ELIZA, was built using regular expression subsitutions, which mimicked a Rogerian psycotherapist in a way that appeared convincing to many people.

[1] Weizenbaum, J. (1966). ELIZA – A computer program forthe study of natural language communication between manand machine.CACM 9(1), 36–45

This was possible because Rogerian psycotherapists often respond with simple questions that don't require a lot of reasoning about what the patient has said. The doc2dial task is much more complex as the agent has to respond to complex customer service queries. In any case, let's see if we can generate some human-like responses to the utterances in the dataset using regular expression substitutions.

The code below responds to utterances containing the phrase 'can you'. For utterances where it does not find the phrase 'can you', it simply replies with 'I do not understand'.

**_TODO 2.5:_** Change the code below to reduce the frequency with which the chatbot says 'I do not understand', and improve the generated responses. Try to make five improvements. You can add new search patterns, and improve the replacement pattern to make the responses more convincing.

**_TODO 2.6:_** Think about the challenge of creating a customer service chatbot using regular expressions. Is it possible? What are the challenges? What kinds of tasks might regular expressions be useful for when building a chatbot?


In [10]:
for match in matches:
    # Pretend each match is an example of user input.
    print("[user] " + match)

    # A regular expression to find the phrase "can you" in the user's input.
    re_can = r".*[Cc]an you"

    if re.search(re_can, match):
        # Substitute "can you" for "yes I can".
        response = re.sub(re_can + r"(.*)", r"Yes, I can\1", match)

        # Substitute "me" for "you".
        response = re.sub(r"me", r"you", response)

        # Print the chatbot's response.
        print("[chatbot] " + response)
    else:
        print("[chatbot] I do not understand.")

[user] can you help me with that
[chatbot] Yes, I can help you with that
[user] Can I do my DMV transactions online
[chatbot] I do not understand.
[user] can sign up for MyDMV for all the online transactions needed
[chatbot] I do not understand.
[user] can I do
[chatbot] I do not understand.
[user] can you tell me again where should I report my new address
[chatbot] Yes, I can tell you again where should I report my new address
[user] Can you tell me more about Traffic points and their cost
[chatbot] Yes, I can tell you more about Traffic points and their cost
[user] Can you tell me more about the traffic points and its cost
[chatbot] Yes, I can tell you more about the traffic points and its cost
[user] can do your transaction online
[chatbot] I do not understand.
[user] Can I leave the state without notifying DMV
[chatbot] I do not understand.
[user] can resolve any previous problem you have in New York State before registering in the new state
[chatbot] I do not understand.
[user] ca

# 3. Tokenisation

Up to now we have been able to work directly with the raw text. However, for most text processing tasks we will need to perform a number of steps to transform the raw text to a suitable format for a model such as a classifier or dialogue system.

One of the key steps is _word tokenisation_, in which we splite a text string into separate pieces or 'tokens', corresponding to words and punctuation. In some languages this is actually quite tricky, and what constitutes a 'word' can have different meanings. Here, we will stick with English.


Let's start with a naïve approach: splitting the sentences based on whitespace. We'll use regular expressions to do it.

The re module provides the re.split() function, which takes a regular expression as its argument and splits the text when it finds a match. The special character '\s' is used to match whitespace characters -- not just spaces, but also tabs, newlines, etc..

**TODO 3.1:** Use re.split() to split the raw text into tokens on whitespace characters. Save the sequence of tokens to a new variable called tokens.


In [11]:
# An example of text that is not tokenized.
text = "If I want to register my vehicle here in new york, I was forewarned that out-of-state insurance can't be accepted? "

tokens = re.split(r"\s", text)

pprint(tokens)

['If',
 'I',
 'want',
 'to',
 'register',
 'my',
 'vehicle',
 'here',
 'in',
 'new',
 'york,',
 'I',
 'was',
 'forewarned',
 'that',
 'out-of-state',
 'insurance',
 "can't",
 'be',
 'accepted?',
 '']


Whitespace tokenisation doesn't handle things like punctuation very well. For example, parentheses '()' are not excluded from the tokens. To see this, run the following code to inspect the non-letter characters in your tokens.


In [12]:
for token in tokens:
    if re.search(r"[^a-zA-Z0-9]", token):
        pprint(token)

'york,'
'out-of-state'
"can't"
'accepted?'


If we start to split the tokens based on any non-letter characters, we can encounter further issues. The punctuation may be informative, so we should not throw it away. Hyphenated words may need to be kept together while contractions like "don't" might need to be split into "do" and "n't".

Luckily, we can make use of existing rule-based tokenizers that deal with these issues:

- Spacy: https://spacy.io/api/tokenizer
- NLTK: https://www.kite.com/python/docs/nltk.word_tokenize

For some domains and languages, tokenisation is not so easy and we may need to construct a regular-experession based approach.

**TODO 3.2:** Refer to the documentation linked above for Spacy or NLTK's word tokeniser, and apply one of them to the raw text. Compare the output to the whitespace tokeniser. Save the tokens to a variable called 'tokens_rulebased'.


In [13]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [14]:
import spacy

nlp = spacy.load("en_core_web_sm")

tokens = [token.text for token in nlp(text)]

pprint(tokens)

['If',
 'I',
 'want',
 'to',
 'register',
 'my',
 'vehicle',
 'here',
 'in',
 'new',
 'york',
 ',',
 'I',
 'was',
 'forewarned',
 'that',
 'out',
 '-',
 'of',
 '-',
 'state',
 'insurance',
 'ca',
 "n't",
 'be',
 'accepted',
 '?']


**TODO 3.3:** Run the code below to see how NLTK has handled the non-letter characters. What does it do with most punctuation marks? When does it include punctuation marks in a token with letters? When does it not split tokens based on punctuation?


In [15]:
for token in tokens:
    if re.search(r"[^a-zA-Z0-9]", token):
        print(token)

,
-
-
n't
?


In the textbook, we also encountered subword tokenization methods, including byte-pair encoding (BPE). These methods have a specific vocabulary of tokens, which they have learned from a large dataset, and will divide the text into tokens from this vocabulary. If they come across an unknown word that is not in the vocabulary, they will divide it by finding smaller sub-word tokens that match part of the unknown word. This may result in a different set of tokens to the NLTK and Spacy methods.

We can test this out using the implementation from HuggingFace's Transformers library:

https://huggingface.co/transformers/tokenizer_summary.html

GPT2 is a famous language model, which has its own tokenizer:


In [16]:
from transformers import GPT2Tokenizer

tokenizer_gpt2 = GPT2Tokenizer.from_pretrained("gpt2")

tokens = tokenizer_gpt2.tokenize(text)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


**TODO 3.4:** Print out some of the tokens and see if you can find any subwords.

There will be some strange symbols that encode whitespaces, which are treated as part of the following word. See if you can work out what they represent.


In [17]:
"""
`Ġ` represents the start of a word, i.e., a token that starts with `Ġ` is a word
or the first subword.
"""
pprint(tokens)

['If',
 'ĠI',
 'Ġwant',
 'Ġto',
 'Ġregister',
 'Ġmy',
 'Ġvehicle',
 'Ġhere',
 'Ġin',
 'Ġnew',
 'Ġy',
 'ork',
 ',',
 'ĠI',
 'Ġwas',
 'Ġfore',
 'warn',
 'ed',
 'Ġthat',
 'Ġout',
 '-',
 'of',
 '-',
 'state',
 'Ġinsurance',
 'Ġcan',
 "'t",
 'Ġbe',
 'Ġaccepted',
 '?',
 'Ġ']


You may also have heard of the BERT model. It uses a similar subword tokenisation method to BPE, called wordpiece. We can also test that out using the HuggingFace Transformers library.

**TODO 3.5:** Use the code below to see if you can find some differences between BERT's wordpiece method and BPE.


In [18]:
from transformers import BertTokenizer

tokenizer_bert = BertTokenizer.from_pretrained("bert-base-uncased")

tokens = tokenizer_bert.tokenize(text)

pprint(tokens)

['if',
 'i',
 'want',
 'to',
 'register',
 'my',
 'vehicle',
 'here',
 'in',
 'new',
 'york',
 ',',
 'i',
 'was',
 'fore',
 '##war',
 '##ned',
 'that',
 'out',
 '-',
 'of',
 '-',
 'state',
 'insurance',
 'can',
 "'",
 't',
 'be',
 'accepted',
 '?']
